This notebook is dedicated to cleaning the data I already have.

<h1>Issue: Line breaks and commas within review text</h1>
Solution: need to go through each review and ensure correct formatting. The parameter encoding="utf-8" in to_csv() should force that.

In [30]:
# set the filename of the CSV to clean
toClean = "lower_threshold.csv"

In [ ]:
import pandas as pd

# Force proper quoting and remove stray characters
df = pd.read_csv(toClean)
df.to_csv(toClean, index=False, encoding="utf-8")

In [31]:
df = pd.read_csv(toClean, engine="python", quotechar='"', on_bad_lines="skip")

# Replace all newlines \r \n with spaces
df = df.replace({r'[\r\n]+': ' '}, regex=True)

df.to_csv(toClean, index=False, encoding="utf-8", quoting=1)

<h1>Issue: Remove unnecessary columns</h1>
We don't really need to keep track of the authorAttribution, flagContentUri, nor googleMapsUri

In [ ]:
df = pd.read_csv(toClean, quoting=1, encoding="utf-8")

df.drop(columns=["authorAttribution", "flagContentUri", "googleMapsUri", "relativePublishTimeDescription"], inplace=True)

df.to_csv(toClean, index=False, encoding="utf-8")

<h1>Issue: column count inconsistent between rows</h1>
This issue stems from the first issue about line breaks and unescaped commas. The following script counts the number of columns in each row. If we see more than one number in set(counts), we know there are some rows in there that don't have the right amount of columns.

In [34]:
import csv
with open(toClean, encoding="utf-8") as f:
    reader = csv.reader(f)
    counts = [len(row) for row in reader]
print(set(counts))

{8}


<h1>Issue: Should convert from wide format to long format</h1>
Originally, the data we collected was in the wide format (one row per restaurant, many columns). This is suboptimal for data visualization purposees. So, we'll reshape our .csv into a long format and keep track of scores for the same restaurant by restaurant name.

In [94]:
df = pd.read_csv(toClean)

score_columns = ["food", "service", "price", "drink", "restaurant", "expectation",
                 "flavor", "staff", "parking", "ambience", "value", "recommendation"]

df_long = pd.melt(df,
                  id_vars=["name", "rating", "text", "originalText", "publishTime"],
                  value_vars=score_columns,
                  var_name="score_type",
                  value_name="score")

df_long.to_csv(toClean)

<h1>Issue: Raw ABSA results have np.float32() wrappers that cannot be ast.literal_eval()'d.</h1>
We'll remove and replace them with regular floats.

In [ ]:
import re
import ast

def parse_results(val):
    if isinstance(val, dict):
        return val
    cleaned = re.sub(r"np\.float32\((.*?)\)", r"\1", val)
    return ast.literal_eval(cleaned)

df['results'] = df['results'].apply(parse_results)

{'food': ('positive', 0.9476525),
 'service': ('positive', 0.9842784),
 'price': ('positive', 0.62646586),
 'parking': ('negative', 0.49753693)}

<h1>Issue: Raw ABSA results are dicts.</h1>
We'll go through the dataframe and convert the results into regular scores.

In [ ]:
import ast
import json

df = pd.read_csv(toClean)

expanded = pd.json_normalize(df['results'])

df = df.drop(columns=['results']).join(expanded)

id                                                                0
name                                    ChIJUaS2CnRskFQRgkVy8FFKcKE
rating                                                            4
text              They have very nice grab n go options but are ...
originalText      {'text': 'They have very nice grab n go option...
publishTime                             2025-05-08T21:42:41.981034Z
food                                          (positive, 0.9476525)
service                                       (positive, 0.9842784)
price                                        (positive, 0.62646586)
parking                                      (negative, 0.49753693)
drink                                                           NaN
restaurant                                                      NaN
expectation                                                     NaN
flavor                                                          NaN
value                                           

In [82]:
def sentiment_to_score(value):
    if isinstance(value, tuple):
        label, score= value
        if label == 'positive':
            if score <= 1:
                score *= 100
            return score
        elif label == 'negative':
            if score <= 1:
                score *= 100
            return -score
        elif label == 'neutral':
            return 0.0
    return value
            

In [89]:
list(df.columns)

['id',
 'name',
 'rating',
 'text',
 'originalText',
 'publishTime',
 'food',
 'service',
 'price',
 'parking',
 'drink',
 'restaurant',
 'expectation',
 'flavor',
 'value',
 'staff',
 'ambience',
 'recommendation']

In [90]:
absa_columns = ['food', 'service', 'ambience', 'staff', 'price', 'value', 'restaurant', 'recommendation',
                'parking', 'drink', 'expectation', 'flavor']
df[absa_columns] = df[absa_columns].applymap(sentiment_to_score)

C:\Users\andy-\AppData\Local\Temp\ipykernel_108800\1754717476.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[absa_columns] = df[absa_columns].applymap(sentiment_to_score)


In [93]:
df.to_csv(toClean)

<h1>Issue: assign human-readable place name, longitude, and latitude to each review by ID</h1>

In [6]:
import pandas as pd

places_df = pd.read_csv("places.csv")
threshold_df = pd.read_csv("lower_threshold.csv")

merged_df = threshold_df.merge(
    places_df[['id', 'displayName', 'longitude', 'latitude']],
    left_on='name',
    right_on='id',
    how='left'
)

# drop the dupllicate ID column
merged_df = merged_df.drop(columns='id')
merged_df.to_csv("lower_threshold_with_coords.csv")